Messing about

In [1]:
import vit_pytorch
import torchvision
import torch
import numpy as np

In [2]:
import dataset

train_data, valid_data = dataset.torch_train()
test_data = dataset.torch_test()

In [3]:
# model = vit_pytorch.ViT(
#     dim=512, 
#     image_size=256, 
#     patch_size=32,
#     num_classes=2,
#     depth=6,heads=12,
#     mlp_dim=1024,
#     channels=1,
#     dropout=0.5
#     ).cuda()


model = vit_pytorch.SimpleViT(
    dim=512, 
    image_size=256, 
    patch_size=32,
    num_classes=2,
    depth=6,heads=12,
    mlp_dim=1024,
    channels=1,
    # dropout=0.5
    ).cuda()


# criterion = torch.nn.BCELoss()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer,step_size=1,gamma=0.8)


In [4]:
train_iter = iter(train_data)
train_size = 0
valid_iter = iter(test_data)
valid_size = 0
for i in train_iter:
    train_size += i[0].shape[0]
for i in valid_iter:
    valid_size += i[0].shape[0]

min_valid_loss = np.inf

history = {'train_loss':[], 'train_acc':[],'valid_loss':[],'valid_acc':[]}
for epoch in range(20):
    train_loss = 0
    train_acc = 0
    train_iter = iter(train_data)
    model.train()
    for batch, labels in train_iter:
        batch, labels = batch.cuda(), labels.cuda()
        optimizer.zero_grad()
        prediction = model(batch)
        loss = criterion(prediction, labels)
        loss.backward()
        optimizer.step()
        acc = sum(torch.argmax(prediction,dim=1) == torch.argmax(labels,dim=1)).cpu().detach().numpy()
        train_acc += acc
        train_loss += loss.cpu().detach().numpy() * len(batch)/train_size
    train_acc /= train_size

    valid_loss = 0
    valid_acc = 0
    model.eval()
    valid_iter = iter(test_data)
    for batch, labels in valid_iter:
        batch, labels = batch.cuda(),labels.cuda()
        prediction = model(batch)
        loss = criterion(prediction, labels)
        acc = sum(torch.argmax(prediction,dim=1) == torch.argmax(labels,dim=1)).cpu().detach().numpy()
        valid_acc += acc
        valid_loss += loss.cpu().detach().numpy()*len(batch)/valid_size
    valid_acc /= valid_size

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['train_acc'].append(valid_acc)
    history['train_loss'].append(valid_loss)

    print(f"Epoch: {epoch+1}\nTrain loss: {train_loss}\nTrain Accuracy: {train_acc}\nValid Loss: {valid_loss}\nValid Accuracy: {valid_acc}\n")

    if min_valid_loss > valid_loss:
        min_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved/best_model.pth')
    

Epoch: 1
Train loss: 0.7287280436785251
Train Accuracy: 0.5257434944237919
Valid Loss: 0.7052050058046977
Valid Accuracy: 0.5225555555555556

Epoch: 2
Train loss: 0.6638504094793892
Train Accuracy: 0.5957713754646841
Valid Loss: 0.7547770191298587
Valid Accuracy: 0.5218888888888888



Test

In [ ]:
test_iter = iter(test_data)
test_acc = 0
test_size = 0
model.eval()
for batch, labels in test_iter:
    batch, labels = batch.cuda(),labels.cuda()
    prediction = model(batch)
    acc = sum(torch.argmax(prediction,dim=1) == torch.argmax(labels,dim=1)).cpu().detach().numpy()
    test_acc += acc
    test_size+= len(batch)

test_acc /= test_size

print(test_acc)

0.6124444444444445


In [6]:
import matplotlib.pyplot as plt

In [1]:
plt.plot(history['train_acc'])

NameError: name 'plt' is not defined

Modules

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import numpy as np




class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches


class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        g = self.projection(patch)
        h = self.position_embedding(positions)

        print(g.shape)
        print(h.shape)
        4 + '4'
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

#ViT Model
def create_model(patch_size = 32, num_patches = 8,projection_dim=64,num_heads=4,transformers=8,transformer_units=[128,64],mlp_head_units=[2048,1024]):
    inputs = layers.Input(shape=(256,256,1))

    patches = Patches(patch_size)(inputs)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(2)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model








